<a href="https://colab.research.google.com/github/vm2781/adversarial_attacks/blob/main/Robust_PGD_Validation_Set_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as DataUtils
import numpy as np
import time
import sys


###  Google Colab doesn't ship with advertorch and we will have to install it ourselves
# !pip install advertorch > /dev/null
# import advertorch
!pip install torchattack > /dev/null
from torchattack import PGD
import torchvision.models as models
import torch.nn as nn


In [ ]:
# Readymade data loading function
DATA_ROOT='./MNISTData/'
def getMNISTDataLoader(batchSize=64, nTrain=59500, nVal=500, nTest=10000):
  # You can use technically use the same transform instance for all 3 sets
  assert (60000 - nVal) == nTrain, 'nTrain + nVal must be equal to 60000'

  valTransform = transforms.Compose([transforms.ToTensor()])

  valSet = datasets.MNIST(root=DATA_ROOT, download=True, train=True, \
                         transform=valTransform)

  indices = np.arange(0, 60000)
  np.random.shuffle(indices)
  valSampler = SubsetRandomSampler(indices[nTrain:])
  valLoader = DataUtils.DataLoader(valSet, batch_size=batchSize, \
                                  sampler=valSampler)
  return valLoader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
adv_model = models.resnet50(num_classes=10, weights=None)
adv_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # ResNet18 output is already 10 classes by default, so no need to change fc layer
    # (it defaults to 1000 classes for ImageNet, but we'll override below)
# adv_model.fc = nn.Linear(512, 10)

model_path = "/content/drive/My Drive/adversarial_mnist/all_saved_models/Poonyapat/resnet50_mnist_normal.pth"

checkpoint = torch.load(model_path, map_location="cpu")
adv_model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
val_loader = getMNISTDataLoader()
print('Notebook will use PyTorch Device: ' + device.upper())


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.1MB/s]

Notebook will use PyTorch Device: CUDA


In [ ]:
all_adv_images = []
all_adv_labels = []
adv_model = adv_model.to(device)

for j, (images, labels) in enumerate(val_loader):
    images, labels = images.to(device), labels.to(device)
    adv_model.eval()

    adversary = PGD(adv_model, eps=0.3, steps=7, random_start=True)
    adv_images = adversary(images, labels)

    all_adv_images.append(adv_images.cpu())
    all_adv_labels.append(labels.cpu())

# Concatenate across all batches
all_adv_images = torch.cat(all_adv_images, dim=0)
all_adv_labels = torch.cat(all_adv_labels, dim=0)

save_path = "/content/drive/My Drive/adversarial_mnist/pgd_eval_dataset.pt"

torch.save(
    {"images": all_adv_images, "labels": all_adv_labels},
    save_path
)

print("Saved to:", save_path)


Saved to: /content/drive/My Drive/adversarial_mnist/pgd_eval_dataset.pt
